In [1]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers


In [2]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29",
    "pCpu",
    "cpus",
    "rss",
    "vmem",
    "rchar",
    "wchar",
    "syscr",
    "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed16', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed22', 'ramspeed23', 'ramspeed24', 'ramspeed25']
x_cols = all_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [ ]:
t = rTable(title="results")
t.add_column("cv")
t.add_column("unknown")
t.add_column("cv score")
t.add_column("unknown score")
for split in getSplits(dF, 1, x_cols, y_cols, cvSize=1, unknownSize=1):
    X_train, y_train, X_test, y_test, X_unknown, y_unknown, X_trans, ukwfs, cvwfs = split
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    # print(cvwfs, ukwfs, model.score(X_test, y_test), model.score(X_unknown, y_unknown))
    t.add_row(cvwfs[0], ukwfs[0], str(model.score(X_test, y_test)), str(model.score(X_unknown, y_unknown)))
rich.print(t)

In [ ]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res = list()
for i in range(100):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.2)
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    res.append(model.score(X_test, y_test))

In [ ]:
rich.print(np.average(res), np.std(res))

In [3]:
taskNames = pds.unique(dF.taskName)


def pickCVTasks():
    cvTasks = list(taskNames)
    totalData = len(dF)
    while len(dF.query("taskName in @cvTasks")) > .3 * totalData:
        cvTasks = list()
        remainingTasks = list(taskNames)
        while len(dF.query("taskName in @cvTasks")) < .2 * totalData:
            pick = np.random.choice(remainingTasks)
            remainingTasks.remove(pick)
            cvTasks.append(pick)
    return cvTasks

In [4]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res = list()
done = set()
for i in range(1000):
    cvTasks = pickCVTasks()
    while set(cvTasks) in done:
        cvTasks = pickCVTasks()
    train = dF.query("taskName not in @cvTasks")
    X_train = train[x_cols]
    X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
    y_train = train[y_cols]
    test = dF.query("taskName in @cvTasks")
    X_test = test[x_cols]
    X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
    y_test = test[y_cols]
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    res.append((cvTasks, model.score(X_test, y_test)))

In [5]:
t = rTable(title="results")
t.add_column("cv tasks")
t.add_column("score")
for r in res[:5]:
    cv, s = r
    t.add_row(", ".join(cv), str(s))
    # print(cv, s)
#rich.print(t)


In [7]:
tmp = dict()
for r in res:
    cv, s = r
    for c in cv:
        if c not in tmp.keys():
            tmp[c] = 0
        tmp[c] = tmp[c] + s/len(cv)
t1 = tmp.keys()
t2 = tmp.values()
t = list(zip(t1, t2))
print(sorted(t, key=lambda x: x[1]))

[('TRIMGALORE', -331.4326160918647), ('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:IVAR_VARIANTS', -100.98670569038404), ('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_BCFTOOLS:PANGOLIN', -99.76757387231247), ('bismark_align', -97.87359313991517), ('FASTQC', -95.80659769696963), ('get_software_versions', -92.88940416133009), ('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:QUAST', -92.55376050802514), ('BWA_INDEX', -90.43817141367096), ('NFCORE_VIRALRECON:ILLUMINA:FASTQC', -89.85159991158324), ('bismark_methXtract', -89.68945051662875), ('GatherBQSRReports', -88.55246560757229), ('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_BCFTOOLS:SNPEFF_SNPSIFT:SNPEFF_ANN', -88.42415493445156), ('ApplyBQSR', -86.03167444368253), ('PHANTOMPEAKQUALTOOLS', -84.92753335248496), ('NFCORE_VIRALRECON:ILLUMINA:KRAKEN2_RUN', -83.11309870444211), ('MERGED_BAM', -82.39876081321508), ('BWA_MEM', -79.85777343603495), ('NFCORE_VIRALRECON:ILLUMINA:PICARD_COLLECTMULTIPLEMETRICS', -79.5385484204419), ('CONSENSUS_PEAKS_DESEQ2', -78.5609786332